In [ ]:
import os
import json
import logging
from dotenv import load_dotenv
import openai

# Load environment variables from .env
load_dotenv()

# Set up logging for chain-of-thought outputs
logging.basicConfig(
    filename='../logs/reasoning.log',  # Adjust path if running from notebook root
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
config_path = '../config/models_config.yaml'
try:
    import yaml
    with open(config_path, 'r') as file:
        model_catalog = yaml.safe_load(file)
except Exception as e:
    # Fallback: define a default catalog if config file is not available
    model_catalog = {
        "openai": ["gpt-4o", "gpt-3.5-turbo"],
        "gemini": [],
        "llama": [],
        "claude": [],
        "deepseek": []
    }
print("Available models:", model_catalog)

In [ ]:
# Make sure your .env file has: OPENAI_API_KEY=your_key_here
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set your OPENAI_API_KEY in the .env file.")

openai.api_key = OPENAI_API_KEY

def query_openai(prompt, model="gpt-4o", log_verbose=True):
    """
    Query the OpenAI model and optionally log verbose chain-of-thought output.
    """
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            # You might add parameters that enable chain-of-thought if available
        )
        # Extract final output
        final_output = response.choices[0].message['content']
        
        # If verbose chain-of-thought info is available (this may vary by model/implementation)
        verbose_info = response.get("verbose", "No verbose chain-of-thought provided.")
        if log_verbose:
            logging.info(f"Prompt: {prompt}\nVerbose Output: {verbose_info}")
        
        return final_output
    except Exception as err:
        logging.error("Error querying OpenAI: " + str(err))
        return f"Error: {err}"

In [ ]:
prompt = "Explain why Paris is called the city of lights."
output = query_openai(prompt, model="gpt-4o")
print("Model Output:\n", output)